In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
# --- 1. Đường dẫn ---
base_dir = Path(r"C:\Downloads\Corn_data_3_region\corn_progress")
out_file = Path(r"C:\Downloads\Corn_data_merged_3_region\corn_progress_all_years_3region.csv")

In [3]:
# --- 2. Bảng ánh xạ vùng ---
REGION_MAP = {
    # Temperate Humid (9)
    "IA":"Temperate Humid","IL":"Temperate Humid","IN":"Temperate Humid",
    "MO":"Temperate Humid","OH":"Temperate Humid","MI":"Temperate Humid",
    "MN":"Temperate Humid","WI":"Temperate Humid","PA":"Temperate Humid",
    # Semi-Arid Continental (6)
    "ND":"Semi-Arid Continental","SD":"Semi-Arid Continental","NE":"Semi-Arid Continental",
    "KS":"Semi-Arid Continental","CO":"Semi-Arid Continental","TX":"Semi-Arid Continental",
    # Subtropical Humid (3)
    "KY":"Subtropical Humid","TN":"Subtropical Humid","NC":"Subtropical Humid"
}


In [4]:
# --- 3. Gộp toàn bộ dữ liệu ---
all_dfs = []

for year_dir in sorted(base_dir.iterdir()):
    if not year_dir.is_dir():
        continue

    for csv_file in sorted(year_dir.glob("*.csv")):
        df = pd.read_csv(csv_file)
        df.columns = [c.strip().lower() for c in df.columns]

        if "state_alpha" in df.columns:
            df["region"] = df["state_alpha"].map(REGION_MAP)
        else:
            print(f" Thiếu cột state_alpha trong {csv_file.name}")

        # --- Sắp lại vị trí cột: đưa region ngay sau state_alpha ---
        if "region" in df.columns and "state_alpha" in df.columns:
            cols = list(df.columns)
            region_pos = cols.index("state_alpha") + 1
            # bỏ region rồi chèn lại sau state_alpha
            cols.remove("region")
            cols.insert(region_pos, "region")
            df = df[cols]

        all_dfs.append(df)


In [5]:
# --- 4. Gộp toàn bộ ---
merged = pd.concat(all_dfs, ignore_index=True)

# --- 5. Ghi file tổng (tạo mới hoặc append) ---
mode, header = ("w", True) if not out_file.exists() else ("a", False)
merged.to_csv(out_file, mode=mode, header=header, index=False, encoding="utf-8-sig")

print(f" Đã gộp xong progress + region")
print(f" File: {out_file}")
print(f" Dòng: {len(merged):,} | Cột: {len(merged.columns)}")
print(" Thứ tự cột:", merged.columns.tolist())


 Đã gộp xong progress + region
 File: C:\Downloads\Corn_data_merged_3_region\corn_progress_all_years_3region.csv
 Dòng: 11,890 | Cột: 13
 Thứ tự cột: ['year', 'state_name', 'state_alpha', 'region', 'week_ending', 'week', 'planted', 'emerged', 'silking', 'doughing', 'dented', 'mature', 'harvested']
